# Solución de ecuaciones
<p><code>Python en Jupyter Notebook</code></p>
<p>Creado por <code>Giancarlo Ortiz</code> para el curso de <code>Métodos Numéricos</code></p>
<style type="text/css">
    .border {
        display: inline-block;
        border: solid 1px rgba(204, 204, 204, 0.4);
        border-bottom-color: rgba(187, 187, 187, 0.4);
        border-radius: 3px;
        box-shadow: inset 0 -1px 0 rgba(187, 187, 187, 0.4);
        background-color: inherit !important;
        vertical-align: middle;
        color: inherit !important;
        font-size: 11px;
        padding: 3px 5px;
        margin: 0 2px;
    }
</style>

## Búsqueda de raíces por métodos abiertos
Los métodos abiertos, a diferencia de los métodos cerrados, calculan en cada iteración una aproximación a la raíz partiendo de un valor inicial y una función auxiliar, sin necesidad de verificar lo que sucede dentro de un intervalo.

## Agenda
1. Generalidades
1. Método de punto fijo
1. Método de Newton Raphson
1. Método de la secante

In [1]:
# Importar módulos al cuaderno de jupyter
import math as m 
import numpy as np
import pylab as plt

# Definir e incluir nuevas funciones al cuaderno
def _buscar_intervalos(fun, ini, fin):
    """ Método para buscar intervalos en los que ocurra cambio de signo.

        ## Parámetros:
            fun (function): función para analizar.
            ini (int): inicio del análisis.
            fin (int): limite final del análisis.
        
        ## Devoluciones:
            I (list): Lista de tuplas con los intervalos donde hay un cero.
    """
    i = ini - 1 # Variable para contar iteraciones
    I = []      # Variable para almacenar los intervalos
    
    while i < fin:
        i += 1
        if fun(i) == 0:
            I.append((i, i))
        elif fun(i) * fun(i+1) < 0:
            I.append((i, i+1))
        else:
            pass
    return I

def _imprimir_paso(paso):
    Iteraciones = len(paso[0])
    Tb = f"| I # | {'Xi':>8} | {'Error Abs':>9} | {'F(Xi)':>9} |\n"
    Tb += "------------------------------------------\n" 
    for i in range(Iteraciones):
        xi = paso[0][i]
        eai = paso[1][i]
        yi = paso[2][i]
        Tb += f"| {i+1:3} | {xi:8.5f} | {eai:9.5f} | {yi:9.5f} |\n"
    
    return print(Tb)


## 1. Métodos Abiertos
---
Los métodos abiertos calculan en cada iteración una aproximación a la raíz partiendo de un valor inicial y una función auxiliar, pero no se ocupan de verificar si esta aproximación genera o no un intervalo que contenga una raíz; como consecuencia de ello en estos métodos no hay garantía de la convergencia.

La función auxiliar usada para predecir la raíz es una fórmula puede desarrollarse como la sustitución de un punto fijo en cada iteración y el valor inicial que se sustituye en la función auxiliar se conoce como semilla.

> **NOTA:** Nótese que para estos métodos no es necesario tener un intervalo, solo es necesaria la semilla, el máximo de iteraciones y la tolerancia aceptada.

## 2. Método de punto fijo
---
Es el primero y más sencillo de los métodos abiertos y se basa en la definición de un punto fijo de forma que para buscar la raíz de una ecuación $f(x)=0$, se genera una función auxiliar de punto fijo $x=g(x)$ y se calcula secuencialmente el valor $x$ de la función partiendo de la semilla $x_o$ introducida al inicio, buscando que en cada iteración el valor de $x$ se acerque al valor de la raíz; continuando hasta que se reduzca el nivel de error a un nivel aceptable, según la tolerancia; o hasta que se supere el número máximo de iteraciones.

> **PUNTO FIJO:** un punto fijo de una función $f(x)$ es un número real p tal que $p = f(p)$

<p align="center">
  <img height="200" src="img/algorithm_open_a.png">
</p>

\begin{align}
\tag{1} f(x) &= 0 \\
\tag{2} f(x) + x &= x \\
\tag{3} g(x) &= x \\
\tag{4} x_n &= g(x_{n-1}) \\
\end{align}

### Ventajas:
* En algunos casos puede converger más rápidamente que los métodos cerrados.
* Útil para entender el funcionamiento de los métodos abiertos.

### Desventajas:
* El método no siempre converge.

In [2]:
# Defino el método del punto fijo
def _fijo(Func, SemX, Imax, Tmax):
    """ Método de la Bisección para encontrar raíces en un intervalo.

        ## Parámetros:
            Func (int): función que depende de una variable.
            SemX (int): Semilla de la solución.
            Imax (int): número máximo de iteraciones.
            Tmax (int): exponente de tolerancia máxima, T = 1e^Tmax.
        
        ## Devoluciones:
            Km (float): valor de x encontrado.
            No (int)  : iteraciones.
            ea (float): error absoluto final
            Li (list): lista de las soluciones, los errores absolutos y la tabla con la evoluvión de iteraciones.
    """
    def g(x):
        y = Func(x) + x
        return y
    
    xo = SemX
    x = []
    ea = []
    y = []
    for i in range(Imax):
        xi = g(xo)
        yi = Func(xi)
        eai = abs((xi - g(xi))/xi)
        x.append(xi)
        ea.append(eai)
        y.append(yi)
        if eai < (10**Tmax): break
        xo = xi

    # Salida del método Xm, No, ea, [S, E, Tb]
    return i, xi, eai, [x, ea, y]


## 3. Método de Newton-Raphson
---
En matemáticas, el método de Newton-Raphson es una variante del método del punto fijo, en este caso la función auxiliar se define de tal forma que linealice la función en las cercanías del valor semilla, y sucesivamente calcular el corte dicha recta como un nuevo valor semilla.

<p align="center">
  <img height="200" src="img/algorithm_open_b.png">
</p>

\begin{align}
\tag{1} g(x) &= x - \frac{f(x)}{f(x)} \\
\tag{2} x_n &= x_{n-1} - \frac{f^{\prime}(x_{n-1})}{f(x_{n-1})} \\
\end{align}

### Ventajas:
* Muy eficiente cuando la ecuación no posee múltiples puntos de inflexión o pendientes prolongadas alrededor de la raíz.
* Útil como aproximación inicial de otros métodos.

### Desventajas:
* El método no siempre converge.
* Es útil definir un valor cercano de la raíz, para evitar puntos criticos en la cercania de la solución.


In [4]:
# Defino el método Newton-Raphson
def _newton(Func, Xmin, Xmax, Imax, Tmax):
    """ Método de la Bisección para encontrar raíces en un intervalo.

        ## Parámetros:
            Func (int): función que depende de una variable.
            Xmin (int): limite inferior de intervalo.
            Xmax (int): limite superior de intervalo.
            Imax (int): número máximo de iteraciones.
            Tmax (int): exponente de tolerancia máxima, T = 1e^Tmax.
        
        ## Devoluciones:
            Km (float): valor de x encontrado.
            No (int)  : iteraciones.
            ea (float): error absoluto final
            Li (list): lista de las soluciones, los errores absolutos y la tabla con la evoluvión de iteraciones.
    """
    

In [3]:
# Ecuación de la altura para el movimiento parabólico
def altura(θ, Vo, ho, t):
    g = 9.8179
    Voy = Vo * np.sin(θ*np.pi/180)
    y = -(1/2) * g * t**2 + Voy * t + ho
    return y

# Renombrar la función
# F = lambda t: altura(45, 12, 3, t) 
F = lambda x: np.cos(x) - x

n1, x1, ea1, paso1  = _fijo(F, 0.7, 50, -4)
_imprimir_paso(paso1)

0.7390537062865034
| I # |       Xi | Error Abs |     F(Xi) |
------------------------------------------
|   1 |  0.76484 |   0.05668 |  -0.04335 |
|   2 |  0.72149 |   0.04065 |   0.02933 |
|   3 |  0.75082 |   0.02623 |  -0.01969 |
|   4 |  0.73113 |   0.01818 |   0.01329 |
|   5 |  0.74442 |   0.01201 |  -0.00894 |
|   6 |  0.73548 |   0.00820 |   0.00603 |
|   7 |  0.74151 |   0.00547 |  -0.00406 |
|   8 |  0.73745 |   0.00371 |   0.00273 |
|   9 |  0.74019 |   0.00249 |  -0.00184 |
|  10 |  0.73834 |   0.00168 |   0.00124 |
|  11 |  0.73958 |   0.00113 |  -0.00084 |
|  12 |  0.73875 |   0.00076 |   0.00056 |
|  13 |  0.73931 |   0.00051 |  -0.00038 |
|  14 |  0.73893 |   0.00035 |   0.00026 |
|  15 |  0.73919 |   0.00023 |  -0.00017 |
|  16 |  0.73902 |   0.00016 |   0.00012 |
|  17 |  0.73913 |   0.00011 |  -0.00008 |
|  18 |  0.73905 |   0.00007 |   0.00005 |



## 4. Metodo de la Secante
---
### a. Ecuaciones lineales
Un caso particular de las ecuaciones algebraicas sucede cuando solo los dos primeros coeficientes son distintos de cero y la solucion para x es unica y trivial.

\begin{align}
a_0 + a_1 x & = 0, \quad a_1 \neq 0 \\
x & = \frac{-a_0}{a_1} \\
\end{align}

### b. Ecuaciones cuadraticas
Un caso particular de las ecuaciones algebraicas sucede cuando solo los tres primeros coeficientes son distintos de cero y las 2 soluciones para x no son compleja.

\begin{align}
a_0 + a_1 x + a_2 x^2 & = 0, \quad a_1 \neq 0 \\
x_1, x_2 & = \frac{-a_1 \pm \sqrt{a_1^2 - 4 a_2 a_0 }}{2 a_2} \\
\end{align}

In [5]:
# Defino el método iterativo de la Secante
def _secante(Func, Xmin, Xmax, Imax, Tmax):
    """ Método de la Bisección para encontrar raíces en un intervalo.

        ## Parámetros:
            Func (int): función que depende de una variable.
            Xmin (int): limite inferior de intervalo.
            Xmax (int): limite superior de intervalo.
            Imax (int): número máximo de iteraciones.
            Tmax (int): exponente de tolerancia máxima, T = 1e^Tmax.
        
        ## Devoluciones:
            Km (float): valor de x encontrado.
            No (int)  : iteraciones.
            ea (float): error absoluto final
            Li (list): lista de las soluciones, los errores absolutos y la tabla con la evoluvión de iteraciones.
    """

---
## Mas Recursos

- [Propagación de errores](https://en.wikipedia.org/wiki/Rounding) (Wikipedia)
- [Método iterativo](https://es.wikipedia.org/wiki/M%C3%A9todo_iterativo) (Wikipedia)
- [Estabilidad numérica](https://es.wikipedia.org/wiki/Estabilidad_num%C3%A9rica) (Wikipedia)
- [Orden de convergencia](https://es.wikipedia.org/wiki/Orden_de_convergencia) (Wikipedia)
